In [ ]:
#import packages
from __future__ import print_function, division
import numpy as np
import random
import torch
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
from torchvision import datasets, models, transforms
from torchvision.transforms import ToPILImage
import torchvision.transforms.functional as tf
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt
from PIL import Image
from torch.utils import data
import time
import math
import os
import copy
import json
import random

In [ ]:
def seed_everything(SEED=42):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True # keep True if all the input have same size.
SEED=42
seed_everything(SEED=SEED)

transformation

In [1]:
def pad(img):
    s = img.size
    w_increase = abs((s[0]-max(s))//2)
    h_increase = abs((s[1]-max(s))//2)
    up_increase = max(s)-s[0]-w_increase
    down_increase = max(s)-s[1]-h_increase
    padding = (w_increase,h_increase,up_increase,down_increase)
    return tf.pad(img, padding)

def mytransform1(image,lable,flip=True,jitter=True):
    image = pad(image)
    mask = pad(lable)
    image = tf.resize(image, (224,224))
    lable = tf.resize(lable, (224,224)) 
    if flip == True:
        
        if random.random() > 0.5:
            image = tf.hflip(image)
            lable = tf.hflip(lable)
        if random.random() > 0.5:
            image = tf.vflip(image)
            lable = tf.vflip(lable)
    if jitter == True:
        if  random.random() > 0.7:
            
            image = tf.adjust_contrast(image,random.uniform(0.9,1.1))
            image = tf.adjust_brightness(image,random.uniform(0.7,1.3))
    image = tf.to_tensor(image)
    lable = np.array(mask)
    lable = torch.as_tensor(lable, dtype=torch.int64)

    image = tf.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
    return image, lable

def mytransform2(image,lable):
    return mytransform1(image,lable,flip=False,jitter=False)
data_transform = {'train': mytransform1, 'val': mytransform2}

customize dataset

In [ ]:
class LipDataset(data.Dataset):
    def __init__(self, file_path=None, name=None, transform=None,number=300):   
        self.file_path = file_path
        self.transform = transform
        self.name = name
        self.number = number
        # initialize
        self.init_all_data(file_path) 
        return None
    def init_all_data(self, file_path):
        
        # record the path
        self.images = []
        self.lables = []
        #because the whole set is too big, I only number to do a trial first
        totallist = [line.rstrip('\n') for line in
                        open(os.path.join(self.file_path, 'TrainVal_pose_annotations', 'lip_'+self.name+'_set.csv'), 'r')]
        for idx in range(self.number):
            tokens = totallist[idx].split(',')
            i = tokens[0].split('.')[0]+'.png'
            image_path = os.path.join(self.file_path, 'TrainVal_images', 'TrainVal_images', self.name+'_images', tokens[0])
            lable_path = os.path.join(self.file_path, 'TrainVal_parsing_annotations', 'TrainVal_parsing_annotations','TrainVal_parsing_annotations',self.name+'_segmentations', i)
            if self.is_valid_image(image_path) and self.is_valid_image(lable_path):
                self.images.append(image_path)
                self.lables.append(lable_path) 
        return None               
    def is_valid_image(self, img_path):
        try:
            # 若读取成功，设valid为True
            i = Image.open(img_path)
            valid = True
        except:
            # 若读取失败，设valid为False
            valid = False
            
        return valid        

    def __getitem__(self, idx):
       #turn image to rgb
        image = Image.open(self.images[idx]).convert('RGB')
        lable = Image.open(self.lables[idx])
        if self.transform:
            image,lable = self.transform(image,lable)

                         
        return image, lable   
    def __len__(self):
        return len(self.images)

load data

In [ ]:
from torch.utils.data import DataLoader
trainsize = 10000
valsize = 300
data_name = {'filepath':'../input/singleperson',
            'train': ('train',trainsize),
             'val': ('val',valsize),
               }

dataset = {x: LipDataset(data_name['filepath'], data_name[x][0], data_transform[x],data_name[x][1] ) for x in ['train', 'val']}
datasize = {x: len(dataset[x]) for x in ['train', 'val']}
dataloader = {}
val_loader = DataLoader(dataset['val'], batch_size=16, shuffle=True, num_workers=2) 
train_loader = DataLoader(dataset['train'], batch_size=16, shuffle=True, num_workers=2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Unet and Unet（mobile net version）

In [ ]:
import torch.nn as nn

from math import sqrt
class Double_Conv_Block(nn.Module):
    def __init__(self, input_channel, output_channel):
        super(Double_Conv_Block, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_channel, output_channel, kernel_size=3, stride=1, padding=1, bias=True),
            #if there is padding=1,then no crop
            nn.BatchNorm2d(output_channel),
            nn.ReLU(inplace=True),
            nn.Conv2d(output_channel, output_channel, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(output_channel),
            nn.ReLU(inplace=True)
        )

    def forward(self, input_channel): 
        return self.conv(input_channel)

class Up_Conv_Block(nn.Module): #Up sampling
    def __init__(self, input_channel, output_channel):
        super(Up_Conv_Block, self).__init__()
        self.up = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv2d(input_channel, output_channel, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(output_channel),
            nn.ReLU(inplace=True)
        )

    def forward(self, input_channel):
        
        return self.up(input_channel)


class U_Net(nn.Module):
    def __init__(self, img_ch=3, output_ch=20):
        #img_ch=3 when RGB
        #output_ch=1 for our project
        super(U_Net, self).__init__()

        self.Maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.Conv1 = Double_Conv_Block(input_channel=img_ch, output_channel=64)
        self.Conv2 = Double_Conv_Block(input_channel=64, output_channel=128)
        self.Conv3 = Double_Conv_Block(input_channel=128, output_channel=256)
        self.Conv4 = Double_Conv_Block(input_channel=256, output_channel=512)
        self.Conv5 = Double_Conv_Block(input_channel=512, output_channel=1024)

        self.Up5 = Up_Conv_Block(input_channel=1024, output_channel=512)
        self.Up_Conv_Block5 = Double_Conv_Block(input_channel=1024, output_channel=512)

        self.Up4 = Up_Conv_Block(input_channel=512, output_channel=256)
        self.Up_Conv_Block4 = Double_Conv_Block(input_channel=512, output_channel=256)

        self.Up3 = Up_Conv_Block(input_channel=256, output_channel=128)
        self.Up_Conv_Block3 = Double_Conv_Block(input_channel=256, output_channel=128)

        self.Up2 = Up_Conv_Block(input_channel=128, output_channel=64)
        self.Up_Conv_Block2 = Double_Conv_Block(input_channel=128, output_channel=64)

        self.Conv_1x1 = nn.Conv2d(64, output_ch, kernel_size=1, stride=1, padding=0)
        
        #initialize weight
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.input_channel
                m.weight.data.normal_(0, sqrt(2. / n))
                nn.init.constant_(m.bias,0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight,1)
                nn.init.constant_(m.bias,0)

    def forward(self, x):
        # downsample 
        x1 = self.Conv1(x)

        x2 = self.Maxpool(x1)
        x2 = self.Conv2(x2)

        x3 = self.Maxpool(x2)
        x3 = self.Conv3(x3)

        x4 = self.Maxpool(x3)
        x4 = self.Conv4(x4)

        x5 = self.Maxpool(x4)
        x5 = self.Conv5(x5)

        # up sample + concat 
        d5 = self.Up5(x5)
        d5 = torch.cat((x4, d5), dim=1)

        d5 = self.Up_Conv_Block5(d5)

        d4 = self.Up4(d5)
        d4 = torch.cat((x3, d4), dim=1)
        d4 = self.Up_Conv_Block4(d4)

        d3 = self.Up3(d4)
        d3 = torch.cat((x2, d3), dim=1)
        d3 = self.Up_Conv_Block3(d3)

        d2 = self.Up2(d3)
        d2 = torch.cat((x1, d2), dim=1)
        d2 = self.Up_Conv_Block2(d2)

        d1 = self.Conv_1x1(d2)

        return d1

In [ ]:
import torch.nn as nn
###the mobile net version of Unet
from math import sqrt
class Double_Conv_Block(nn.Module):
    def __init__(self, input_channel, output_channel):
        super(Double_Conv_Block, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_channel, output_channel, kernel_size=3, stride=1, padding=1, bias=True),
            #if there is padding=1,then no crop
            nn.BatchNorm2d(output_channel),
            nn.ReLU(inplace=True),
            nn.Conv2d(output_channel, output_channel, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(output_channel),
            nn.ReLU(inplace=True)
        )

    def forward(self, input_channel): 
        return self.conv(input_channel)

class Up_Conv_Block(nn.Module): #Up sampling
    def __init__(self, input_channel, output_channel):
        super(Up_Conv_Block, self).__init__()
        self.up = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv2d(input_channel, output_channel, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(output_channel),
            nn.ReLU(inplace=True)
        )

    def forward(self, input_channel):
        
        return self.up(input_channel)


class U_Net_mobile(nn.Module):
    def __init__(self, img_ch=3, output_ch=20):
        #img_ch=3 when RGB
        #output_ch=1 for our project
        super(U_Net_mobile, self).__init__()

        self.Maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.Conv1 = Double_Conv_Block(input_channel=img_ch, output_channel=64)
        self.Conv2 = Double_Conv_Block(input_channel=64, output_channel=128)
        self.Conv3 = Double_Conv_Block(input_channel=128, output_channel=256)
        self.Conv4 = Double_Conv_Block(input_channel=256, output_channel=512)
        self.Conv5 = Double_Conv_Block(input_channel=512, output_channel=1024)

        self.Up5 = Up_Conv_Block(input_channel=1024, output_channel=512)
        self.Up_Conv_Block5 = Double_Conv_Block(input_channel=1024, output_channel=512)

        self.Up4 = Up_Conv_Block(input_channel=512, output_channel=256)
        self.Up_Conv_Block4 = Double_Conv_Block(input_channel=512, output_channel=256)

        self.Up3 = Up_Conv_Block(input_channel=256, output_channel=128)
        self.Up_Conv_Block3 = Double_Conv_Block(input_channel=256, output_channel=128)

        self.Up2 = Up_Conv_Block(input_channel=128, output_channel=64)
        self.Up_Conv_Block2 = Double_Conv_Block(input_channel=128, output_channel=64)

        self.Conv_1x1 = nn.Conv2d(64, output_ch, kernel_size=1, stride=1, padding=0)
        
       #initialize weight
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.input_channel
                m.weight.data.normal_(0, sqrt(2. / n))
                nn.init.constant_(m.bias,0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight,1)
                nn.init.constant_(m.bias,0)


    def forward(self, x):
        # downsample 
        x1 = self.Conv1(x)

        x2 = self.Maxpool(x1)
        x2 = self.Conv2(x2)

        x3 = self.Maxpool(x2)
        x3 = self.Conv3(x3)

        x4 = self.Maxpool(x3)
        x4 = self.Conv4(x4)

        x5 = self.Maxpool(x4)
        x5 = self.Conv5(x5)

        # up sample + concat 
        d5 = self.Up5(x5)
        d5 = torch.cat((x4, d5), dim=1)

        d5 = self.Up_Conv_Block5(d5)

        d4 = self.Up4(d5)
        d4 = torch.cat((x3, d4), dim=1)
        d4 = self.Up_Conv_Block4(d4)

        d3 = self.Up3(d4)
        d3 = torch.cat((x2, d3), dim=1)
        d3 = self.Up_Conv_Block3(d3)

        d2 = self.Up2(d3)
        d2 = torch.cat((x1, d2), dim=1)
        d2 = self.Up_Conv_Block2(d2)

        d1 = self.Conv_1x1(d2)

        return d1

summary module

In [ ]:
from torch.autograd import Variable

from collections import OrderedDict
import numpy as np


def summary(model, input_size, batch_size=-1, device=torch.device('cuda:0'), dtypes=None):
    result, params_info = summary_string(
        model, input_size, batch_size, device, dtypes)
    print(result)

    return params_info


def summary_string(model, input_size, batch_size=-1, device=torch.device('cuda:0'), dtypes=None):
    if dtypes == None:
        dtypes = [torch.FloatTensor]*len(input_size)

    summary_str = ''

    def register_hook(module):
        def hook(module, input, output):
            class_name = str(module.__class__).split(".")[-1].split("'")[0]
            module_idx = len(summary)

            m_key = "%s-%i" % (class_name, module_idx + 1)
            summary[m_key] = OrderedDict()
            summary[m_key]["input_shape"] = list(input[0].size())
            summary[m_key]["input_shape"][0] = batch_size
            if isinstance(output, (list, tuple)):
                summary[m_key]["output_shape"] = [
                    [-1] + list(o.size())[1:] for o in output
                ]
            else:
                summary[m_key]["output_shape"] = list(output.size())
                summary[m_key]["output_shape"][0] = batch_size

            params = 0
            if hasattr(module, "weight") and hasattr(module.weight, "size"):
                params += torch.prod(torch.LongTensor(list(module.weight.size())))
                summary[m_key]["trainable"] = module.weight.requires_grad
            if hasattr(module, "bias") and hasattr(module.bias, "size"):
                params += torch.prod(torch.LongTensor(list(module.bias.size())))
            summary[m_key]["nb_params"] = params

        if (
            not isinstance(module, nn.Sequential)
            and not isinstance(module, nn.ModuleList)
        ):
            hooks.append(module.register_forward_hook(hook))

    # multiple inputs to the network
    if isinstance(input_size, tuple):
        input_size = [input_size]

    # batch_size of 2 for batchnorm
    x = [torch.rand(2, *in_size).type(dtype).to(device=device)
         for in_size, dtype in zip(input_size, dtypes)]

    # create properties
    summary = OrderedDict()
    hooks = []

    # register hook
    model.apply(register_hook)

    # make a forward pass
    # print(x.shape)
    model(*x)

    # remove these hooks
    for h in hooks:
        h.remove()

    summary_str += "----------------------------------------------------------------" + "\n"
    line_new = "{:>20}  {:>25} {:>15}".format(
        "Layer (type)", "Output Shape", "Param #")
    summary_str += line_new + "\n"
    summary_str += "================================================================" + "\n"
    total_params = 0
    total_output = 0
    trainable_params = 0
    for layer in summary:
        # input_shape, output_shape, trainable, nb_params
        line_new = "{:>20}  {:>25} {:>15}".format(
            layer,
            str(summary[layer]["output_shape"]),
            "{0:,}".format(summary[layer]["nb_params"]),
        )
        total_params += summary[layer]["nb_params"]

        total_output += np.prod(summary[layer]["output_shape"])
        if "trainable" in summary[layer]:
            if summary[layer]["trainable"] == True:
                trainable_params += summary[layer]["nb_params"]
        summary_str += line_new + "\n"

    # assume 4 bytes/number (float on cuda).
    total_input_size = abs(np.prod(sum(input_size, ()))
                           * batch_size * 4. / (1024 ** 2.))
    total_output_size = abs(2. * total_output * 4. /
                            (1024 ** 2.))  # x2 for gradients
    total_params_size = abs(total_params * 4. / (1024 ** 2.))
    total_size = total_params_size + total_output_size + total_input_size

    summary_str += "================================================================" + "\n"
    summary_str += "Total params: {0:,}".format(total_params) + "\n"
    summary_str += "Trainable params: {0:,}".format(trainable_params) + "\n"
    summary_str += "Non-trainable params: {0:,}".format(total_params -
                                                        trainable_params) + "\n"
    summary_str += "----------------------------------------------------------------" + "\n"
    summary_str += "Input size (MB): %0.2f" % total_input_size + "\n"
    summary_str += "Forward/backward pass size (MB): %0.2f" % total_output_size + "\n"
    summary_str += "Params size (MB): %0.2f" % total_params_size + "\n"
    summary_str += "Estimated Total Size (MB): %0.2f" % total_size + "\n"
    summary_str += "----------------------------------------------------------------" + "\n"
    # return summary
    return summary_str, (total_params, trainable_params)

In [ ]:
modelcross = U_Net().cuda()
#modelcross=U_Net_mobile().cuda()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
summary(modelcross,(3, 224, 224),device=device)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(modelcross.parameters(), lr=0.001)###


iou loss

In [ ]:
def Iou(logits, target,n_class=20,device, smooth=1):

    logits = logits.cpu().detach()
    logits = torch.softmax(logits,dim=1)
    pred = torch.max(logits,dim=1)[1].view(-1)
    target = target.cpu().detach().view(-1)      
    accs=np.zeros(n_class)
    for i in range(n_class):        
        p = pred==i
        t = target==i
        intersection = (p * t).sum().numpy()
        union = (p + t).sum().numpy()        
        accs[i]=(intersection + smooth)/(union + smooth)
    return np.mean(accs)

train process

In [ ]:
EPOCH=30
train_num = datasize['train']
val_num = datasize['val']
crosslosses=[]#####
val_accs=[]
train_accs=[]
for epoch in range(EPOCH):
    # train
    modelcross.train()####
    

    running_loss = 0.0
    train_acc=[]
    running_acc = 0.0
    for step, (images, labels) in enumerate(train_loader, start=0):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        logits = modelcross(images)####
        loss = criterion(logits, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()* images.size(0)

        
        running_acc += Iou(preds,labels,20,device)* images.size(0)
        train_accurate = running_acc / train_num
        
        train_acc.append(train_accurate)
        # print train process
        rate = (step+1)/len(train_loader)
        a = "*" * int(rate * 50)
        b = "." * int((1 - rate) * 50)
        print("\rtrain loss: {:^3.0f}%[{}->{}]{:.4f}".format(int(rate*100), a, b, loss), end="")
    print()
    train_accs.append(train_acc[-1])


    # validate
    torch.cuda.empty_cache()
    modelcross.eval()########
    acc = 0.0  # accumulate accurate number / epoch
    with torch.no_grad():
        for (val_images, val_labels) in val_loader:
            val_images, val_labels = val_images.to(device), val_labels.to(device)
            outputs = modelcross(val_images)  # eval model only have last output layer
            ###
            
            acc += Iou(outputs,val_labels,20,device)
        val_accurate = acc /val_num
        val_accs.append(val_accurate)
   
        print('[epoch %d] train_loss: %.4f  test_accuracy: %.4f train_accuracy: %.4f   ' %
              (epoch + 1, running_loss / train_num val_accurate,train_accs[-1]))
        crosslosses.append(running_loss / train_num)####

print('Finished Training')

In [ ]:
x=np.arange(len(crosslosses))
plt.plot(x,crosslosses,label='cross entropy')
plt.title('Loss ')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.show()

In [ ]:
x=np.arange(len(train_accs))
plt.plot(x,train_accs,label='train')
plt.plot(x,val_accs,label='validate')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

visualize the output

In [ ]:
colormap = [
    0,   0,   0,
   50,   0,   0,   
    80,  50,  0,     
    30,   30, 90,   
    0,   0, 128,     
    25,   100, 15,   
    20, 148, 128,     
    168, 168, 168,   
    90,   20,   20,   
    192,   15,   15,   
    55, 125,   50,    
    175, 180,   80,   
    60,   5, 125,    
    150,   10, 90,   
    32, 55, 123,    
    192, 128, 125,   
    55,  64,   10,     
    128,  78,   0,   
    10, 220,   80,     
    125, 75,   45,    
]

In [ ]:
visualset =  torch.utils.data.Subset(dataset['val'], np.arange(4))
visualloader = DataLoader(visualset, batch_size=4, shuffle=False)

for i in range(4):    
    image, lable = visualloader[i]
    plt.subplots(i,3,0)
    image_show = np.transpose(image_show.numpy(),(1,2,0))
    plt.imshow(image_show)
    
    plt.subplots(i,3,1)
    lable_show = torch.as_tensor(lable, dtype=torch.uint8)
    lable_show = ToPILImage()(lable_show)
    lable_show.putpalette(colormap)
    plt.imshow(lable_show)
    
    plt.subplots(i,3,2)
    image = torch.unsqueeze(image,dim=0).to(device)
    lable = lable.to(device)
    output = modelcross(image)
    pred = torch.max(output, 1)[1].cpu().clone()
    pred = torch.as_tensor(pred, dtype=torch.uint8)
    pred = ToPILImage()(pred)
    pred.putpalette(colormap)
    plt.imshow(pred)
    